In [3]:
# | default_exp tools.fs_read

In [2]:
# | export
import os
import re
import time
import logging
from pathlib import Path
from typing import Dict, List, Any, Optional
import fnmatch
import json
from pydantic import BaseModel, field_validator, Field, ValidationInfo, ValidationError
from enum import Enum
from agentic.tools.base import create_success_response, create_error_response, extract_validation_error
from agentic.tools.base import BaseTool, ToolMetadata, ToolCategory

# Set up logging with a clear format
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

class ToolCallMode(str, Enum):
    DISCOVER = "discover"
    EXTRACT = "extract"

class FsReadOperation(BaseModel):
    mode: ToolCallMode
    path: str = Field(
        ...,
        description="Specific file or directory path to operate on, e.g., 'src' or 'app.py'. Relative to project root if not absolute."
    )
    query: Optional[str] = Field(
        None,
        description="Search query as regex for DISCOVER (file name) or EXTRACT (content). Required for EXTRACT."
    )
    file_pattern: str = Field(
        "*",
        description="Glob filter for files, e.g., '*.py|*.ipynb' or '*.go|*.rs'."
    )
    max_depth: Optional[int] = Field(
        10,
        ge=1,
        description="Maximum recursion depth for DISCOVER mode. Value must be >= 1"
    )
    max_files: Optional[int] = Field(
        50,
        ge=1,
        description="Maximum number of files to return in DISCOVER mode"
    )

    @field_validator("path")
    @classmethod
    def validate_path(cls, value: str, info: ValidationInfo) -> str:
        try:
            project_root = os.getcwd()
            if not value:
                value = project_root
                logger.info(f"Path empty; defaulting to project root: {value}")
            path_obj = Path(value).resolve()
            if not path_obj.exists():
                raise ValueError(f"Path does not exist: {value}")
            if not str(path_obj).startswith(project_root):
                raise ValueError(f"Path is outside project root: {value}; explicit permission required")
            return str(path_obj)
        except ValueError as e:
            logger.error(f"Path validation failed: {str(e)}")
            raise ValueError(f"Invalid path: {str(e)}")
        except Exception as e:
            logger.error(f"Unexpected error during path validation: {type(e).__name__} - {str(e)}")
            raise ValueError(f"Unexpected path validation error: {type(e).__name__} - {str(e)}")

    @field_validator("query")
    @classmethod
    def validate_query(cls, value: Any, info: ValidationInfo) -> Any:
        mode = info.data.get("mode")
        if mode == ToolCallMode.EXTRACT and value is None:
            raise ValueError("Query is required for EXTRACT mode to perform regex matching")
        if mode == ToolCallMode.DISCOVER and value is None:
            return ""
        if value:
            try:
                re.compile(value)
            except re.error as e:
                logger.error(f"Invalid regex query: {value} ({str(e)})")
                raise ValueError(f"Invalid regex query: {value} ({str(e)})")
        return value

    @field_validator("file_pattern")
    @classmethod
    def validate_file_pattern(cls, value: Any) -> Any:
        if not isinstance(value, str):
            raise ValueError("file_pattern must be a string")
        try:
            for pattern in value.split('|'):
                fnmatch.fnmatch("test.txt", pattern)
        except Exception as e:
            logger.error(f"Invalid glob pattern: {value} ({str(e)})")
            raise ValueError(f"Invalid glob pattern: {value} ({str(e)})")
        return value

class FsReadParams(BaseModel):
    operations: List[FsReadOperation]

class FsReadTool(BaseTool):
    def __init__(self, log_level: str = "INFO"):
        metadata = ToolMetadata(
            name="fs_read",
            description="Read filesystem with regex search and exclusions, supporting file discovery or content extraction",
            category=ToolCategory.FILESYSTEM
        )
        super().__init__(metadata)
        logging.getLogger().setLevel(getattr(logging, log_level, logging.INFO))
        self.project_root = os.getcwd()
        self.exclusion_patterns = [
            ".*", "*.pyc", "*.o", "*.obj", "*.class", "*.exe", "*.dll", "*.so",
            "*.lock", "node_modules/*", "dist/*", "build/*", "__pycache__/*",
            "*.bin", "*.zip", "*.tar.gz", "*.log"
        ]
        self._load_gitignore()

    def _load_gitignore(self) -> None:
        """Load .gitignore patterns to exclude files."""
        gitignore_path = Path(self.project_root) / ".gitignore"
        try:
            if gitignore_path.exists():
                with open(gitignore_path, 'r', encoding='utf-8', errors='ignore') as f:
                    for line in f:
                        line = line.strip()
                        if line and not line.startswith('#'):
                            self.exclusion_patterns.append(line)
            else:
                logger.debug(f".gitignore file does not exist at {gitignore_path}")
        except OSError as e:
            logger.debug(f"Failed to open .gitignore at {gitignore_path}: {type(e).__name__} - {str(e)}")
        except UnicodeDecodeError as e:
            logger.debug(f"Failed to decode .gitignore at {gitignore_path}: {type(e).__name__} - {str(e)}")
        except Exception as e:
            logger.debug(f"Unexpected error reading .gitignore at {gitignore_path}: {type(e).__name__} - {str(e)}")

    def _is_excluded(self, rel_path: str) -> bool:
        """Check if a path matches exclusion patterns."""
        try:
            return any(fnmatch.fnmatch(rel_path, p) for p in self.exclusion_patterns)
        except Exception as e:
            logger.error(f"Error checking exclusion for path {rel_path}: {type(e).__name__} - {str(e)}")
            return True  # Conservatively exclude on error

    def _get_file_info(self, path: str) -> Dict[str, Any]:
        """Get metadata about a file, including size and binary status."""
        path_obj = Path(path)
        try:
            if not path_obj.exists():
                return {"error": f"File does not exist: {path}"}
            stat = path_obj.stat()
        except OSError as e:
            return {"error": f"Failed to get file stats for {path}: {type(e).__name__} - {str(e)}"}
        except Exception as e:
            return {"error": f"Unexpected error getting stats for {path}: {type(e).__name__} - {str(e)}"}

        try:
            rel_path = os.path.relpath(path, self.project_root)
            if self._is_excluded(rel_path):
                return {"error": f"File is excluded by patterns: {path}"}
        except ValueError as e:
            return {"error": f"Failed to compute relative path for {path}: {type(e).__name__} - {str(e)}"}

        file_size = stat.st_size
        is_binary = False
        lines = 0

        try:
            with open(path, 'rb') as f:
                chunk = f.read(2048)
            if b'\0' in chunk:
                is_binary = True
            else:
                chunk.decode('utf-8')
                with open(path, 'r', encoding='utf-8', errors='ignore') as f_text:
                    lines = sum(1 for _ in f_text)
        except OSError as e:
            return {"error": f"Failed to open file {path}: {type(e).__name__} - {str(e)}"}
        except UnicodeDecodeError:
            is_binary = True
            lines = 0
        except Exception as e:
            return {"error": f"Unexpected error reading file {path}: {type(e).__name__} - {str(e)}"}

        return {
            "size": file_size,
            "is_binary": is_binary,
            "file_type": path_obj.suffix.lower(),
            "is_large": file_size > 1024 * 1024,
            "lines": lines if not is_binary else 0
        }

    def _build_tree(self, rel_paths: List[str]) -> str:
        """Build a tree representation of file paths."""
        try:
            tree = {}
            for rel_path in rel_paths:
                parts = rel_path.split(os.sep)
                current = tree
                for part in parts[:-1]:
                    if part not in current:
                        current[part] = {}
                    current = current[part]
                if parts:
                    current[parts[-1]] = {}

            def print_tree(node, prefix: str = "") -> List[str]:
                lines = []
                items = sorted(node.keys())
                for i, item in enumerate(items):
                    is_last = i == len(items) - 1
                    connector = "└── " if is_last else "├── "
                    lines.append(prefix + connector + item)
                    sub_prefix = prefix + ("    " if is_last else "│   ")
                    lines.extend(print_tree(node[item], sub_prefix))
                return lines

            tree_lines = print_tree(tree)
            return "\n".join(tree_lines) if tree_lines else "No files found"
        except Exception as e:
            logger.error(f"Failed to build file tree: {type(e).__name__} - {str(e)}")
            return "Error: Failed to build file tree structure"

    def _discover_files(self, path: str, file_pattern: str, query: str, max_depth: int, max_files: int, is_suggestion: bool = False) -> str:
        """Discover files matching patterns and query, returning a tree structure or suggestions."""
        start_time = time.time()
        path_obj = Path(path)
        suggestions = []

        # Handle non-existent path by listing files in parent directory
        try:
            if not path_obj.exists():
                parent_path = path_obj.parent if path_obj.parent.exists() else Path.cwd()
                # Use broad file pattern for suggestions
                suggestion_result = self._discover_files(
                    str(parent_path), "*", "", max_depth=2, max_files=10, is_suggestion=True
                )
                try:
                    suggestion_data = json.loads(suggestion_result)
                    if suggestion_data.get("data"):
                        suggestions = suggestion_data["data"][0].split("\n") if suggestion_data["data"] else []
                    message = f"The path doesn't exist: {path}. The directory contains these files; check for potential candidates: {', '.join(suggestions[:10]) if suggestions else 'No files found'}"
                    return json.dumps({
                        "data": [],
                        "message": message,
                        "suggestions": suggestions[:10]
                    })
                except json.JSONDecodeError:
                    return json.dumps({
                        "data": [],
                        "message": f"The path doesn't exist: {path}. Failed to generate suggestions."
                    })
            if not path_obj.is_dir():
                return json.dumps({
                    "data": [],
                    "message": f"The path is not a directory: {path}. Please provide a valid directory path."
                })
        except Exception as e:
            return json.dumps({
                "data": [],
                "message": f"Failed to check path: {type(e).__name__} - {str(e)}"
            })

        try:
            query_pattern = re.compile(query, re.IGNORECASE) if query else None
        except re.error as e:
            return json.dumps({
                "data": [],
                "message": f"Invalid regex query '{query}': {str(e)}. Ensure the regex pattern is valid."
            })

        try:
            patterns = [fnmatch.translate(p) for p in file_pattern.split('|')]
            pattern_regexes = [re.compile(p) for p in patterns]
        except Exception as e:
            return json.dumps({
                "data": [],
                "message": f"Failed to compile file patterns: {type(e).__name__} - {str(e)}. Check the file_pattern syntax."
            })

        candidates = []
        errors = []

        def _collect_with_walk(dir_path: str, current_depth: int, file_count: List[int]) -> None:
            if current_depth > max_depth or file_count[0] >= max_files:
                return
            try:
                for entry in os.scandir(dir_path):
                    rel_path = entry.name
                    try:
                        rel_path = str(Path(entry.path).relative_to(self.project_root))
                    except ValueError:
                        pass
                    if self._is_excluded(rel_path):
                        continue
                    if entry.is_file() and any(p.match(entry.name) for p in pattern_regexes):
                        if query_pattern and not query_pattern.search(entry.name):
                            continue
                        candidates.append(entry.path)
                        file_count[0] += 1
                    if entry.is_dir():
                        _collect_with_walk(entry.path, current_depth + 1, file_count)
            except (OSError, PermissionError) as e:
                errors.append(f"Failed to scan directory {dir_path}: {type(e).__name__} - {str(e)}")
            except Exception as e:
                errors.append(f"Unexpected error scanning directory {dir_path}: {type(e).__name__} - {str(e)}")

        file_count = [0]
        _collect_with_walk(str(path_obj), 0, file_count)

        if not candidates:
            # Generate suggestions by listing files in the directory
            suggestion_result = self._discover_files(
                path, "*", "", max_depth=2, max_files=10, is_suggestion=True
            )
            try:
                suggestion_data = json.loads(suggestion_result)
                suggestions = suggestion_data["data"][0].split("\n") if suggestion_data["data"] else []
                message = f"No files matched the criteria in directory {path}. The directory contains these files; check for potential candidates: {', '.join(suggestions[:10]) if suggestions else 'No files found'}"
                if errors:
                    message += f" Errors encountered: {'; '.join(errors)}"
                return json.dumps({
                    "data": [],
                    "message": message,
                    "suggestions": suggestions[:10]
                })
            except json.JSONDecodeError:
                return json.dumps({
                    "data": [],
                    "message": f"No files matched the criteria in directory {path}. Failed to generate suggestions."
                })

        try:
            candidates.sort()  # Sort alphabetically
            selected_paths = candidates[:max_files]
            rel_paths = [os.path.relpath(p, path) for p in selected_paths]
            tree_str = self._build_tree(rel_paths)
        except Exception as e:
            return json.dumps({
                "data": [],
                "message": f"Failed to process discovered files: {type(e).__name__} - {str(e)}. Ensure file paths are valid."
            })

        logger.info(f"Discovered {len(selected_paths)} files in {time.time() - start_time:.2f}s")
        return json.dumps({
            "data": [tree_str],
            "message": "Operation completed successfully"
        })

    def _extract_content(self, path: str, query: str, file_pattern: str) -> str:
        """Extract content from files matching query and pattern."""
        path_obj = Path(path)
        try:
            if not path_obj.exists():
                parent_path = path_obj.parent if path_obj.parent.exists() else Path.cwd()
                suggestion_result = self._discover_files(
                    str(parent_path), "*", "", max_depth=2, max_files=10, is_suggestion=True
                )
                try:
                    suggestion_data = json.loads(suggestion_result)
                    suggestions = suggestion_data["data"][0].split("\n") if suggestion_data["data"] else []
                    message = f"The path doesn't exist: {path}. The directory contains these files; check for potential candidates: {', '.join(suggestions[:10]) if suggestions else 'No files found'}"
                    return json.dumps({
                        "data": [],
                        "message": message,
                        "suggestions": suggestions
                    })
                except json.JSONDecodeError:
                    return json.dumps({
                        "data": [],
                        "message": f"The path doesn't exist: {path}. Failed to generate suggestions."
                    })
        except Exception as e:
            return json.dumps({
                "data": [],
                "message": f"Failed to check path existence: {type(e).__name__} - {str(e)}"
            })

        if path_obj.is_file():
            try:
                rel_path = os.path.relpath(path, self.project_root)
                if self._is_excluded(rel_path):
                    return json.dumps({
                        "data": [],
                        "message": f"The path is excluded by patterns: {path}. The directory contains these files; check for potential candidates: []"
                    })
                content = self._extract_from_file(path, query)
                if content == "No matches found":
                    suggestion_result = self._discover_files(
                        str(path_obj.parent), "*", "", max_depth=2, max_files=10, is_suggestion=True
                    )
                    try:
                        suggestion_data = json.loads(suggestion_result)
                        suggestions = suggestion_data["data"][0].split("\n") if suggestion_data["data"] else []
                        message = f"No matches found for query '{query}' in {path}. The directory contains these files; check for potential candidates: {', '.join(suggestions[:10]) if suggestions else 'No files found'}"
                        return json.dumps({
                            "data": [],
                            "message": message
                        })
                    except json.JSONDecodeError:
                        return json.dumps({
                            "data": [],
                            "message": f"No matches found for query '{query}' in {path}. Failed to generate suggestions."
                        })
                return json.dumps({
                    "data": [{"file": rel_path, "snippet": content}],
                    "message": "Operation completed successfully"
                })
            except ValueError as e:
                return json.dumps({
                    "data": [],
                    "message": f"Failed to compute relative path for {path}: {type(e).__name__} - {str(e)}"
                })
            except Exception as e:
                return json.dumps({
                    "data": [],
                    "message": f"Failed to process file {path}: {type(e).__name__} - {str(e)}"
                })
        else:
            errors = []
            file_errors = []
            snippets = []

            def onerror(e):
                filename = getattr(e, 'filename', path)
                errors.append(f"Failed to scan directory {filename}: {type(e).__name__} - {str(e)}")

            try:
                for root, dirs, files in os.walk(path, onerror=onerror):
                    for file in files:
                        if any(fnmatch.fnmatch(file, p) for p in file_pattern.split('|')):
                            full_path = os.path.join(root, file)
                            try:
                                rel_path = os.path.relpath(full_path, self.project_root)
                                if not self._is_excluded(rel_path):
                                    file_snip = self._extract_from_file(full_path, query)
                                    if file_snip.startswith("Error"):
                                        file_errors.append(f"{file_snip} in file {rel_path}")
                                    elif file_snip.startswith("[Binary"):
                                        pass
                                    else:
                                        if file_snip and file_snip != "No matches found":
                                            snippets.append({"file": rel_path, "snippet": file_snip[:1000]})
                            except ValueError as e:
                                file_errors.append(f"Failed to compute relative path for {full_path}: {type(e).__name__} - {str(e)}")
                            except Exception as e:
                                file_errors.append(f"Unexpected error processing file {full_path}: {type(e).__name__} - {str(e)}")
            except Exception as e:
                return json.dumps({
                    "data": [],
                    "message": f"Failed to traverse directory {path}: {type(e).__name__} - {str(e)}"
                })

            if not snippets:
                suggestion_result = self._discover_files(
                    path, "*", "", max_depth=2, max_files=10, is_suggestion=True
                )
                try:
                    suggestion_data = json.loads(suggestion_result)
                    suggestions = suggestion_data["data"][0].split("\n") if suggestion_data["data"] else []
                    message = f"No matches found for query '{query}' in directory {path}. The directory contains these files; check for potential candidates: {', '.join(suggestions[:10]) if suggestions else 'No files found'}"
                    if file_errors:
                        message += f" File errors: {'; '.join(file_errors[:3])}"
                    if errors:
                        message += f" Directory scan errors: {'; '.join(errors)}"
                    return json.dumps({
                        "data": [],
                        "message": message
                    })
                except json.JSONDecodeError:
                    return json.dumps({
                        "data": [],
                        "message": f"No matches found for query '{query}' in directory {path}. Failed to generate suggestions."
                    })

            try:
                content = [{"file": s["file"], "snippet": s["snippet"]} for s in snippets]
                if len(json.dumps(content)) > 16 * 1024:
                    content = content[:10]
                    message = "Operation completed successfully, but output was truncated due to size limits."
                else:
                    message = "Operation completed successfully"
                if file_errors:
                    message += f" File errors: {'; '.join(file_errors[:3])}"
                if errors:
                    message += f" Directory scan errors: {'; '.join(errors)}"
                return json.dumps({
                    "data": content,
                    "message": message
                }, ensure_ascii=False)
            except Exception as e:
                return json.dumps({
                    "data": [],
                    "message": f"Failed to serialize extracted content to JSON: {type(e).__name__} - {str(e)}"
                })

    def _extract_from_file(self, file_path: str, query: str) -> str:
        """Extract content from a single file with regex matching."""
        file_info = self._get_file_info(file_path)
        if "error" in file_info:
            return f"Error: {file_info['error']}"
        if file_info["is_binary"]:
            return f"[Binary file: {file_info['size']} bytes]"

        try:
            pattern = re.compile(query, re.IGNORECASE | re.MULTILINE) if query else None
        except re.error as e:
            return f"Error: Invalid regex pattern in query for file {file_path}: {type(e).__name__} - {str(e)}"

        try:
            with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                lines = f.readlines()
        except OSError as e:
            return f"Error reading file {file_path}: {type(e).__name__} - {str(e)}"
        except UnicodeDecodeError as e:
            return f"Error decoding file {file_path}: {type(e).__name__} - {str(e)}"
        except Exception as e:
            return f"Error: Unexpected error reading file {file_path}: {type(e).__name__} - {str(e)}"

        try:
            if not query:
                content = "".join(lines)
                if len(content) > 1024:
                    content = content[:1024] + "... [truncated]"
                return content + f"\n--- File Info: {file_info['lines']} lines, {file_info['size']} bytes ---"
            matches = []
            for line_num, line in enumerate(lines, 1):
                if pattern and pattern.search(line):
                    clean_line = line.strip().replace('"', '\\"').replace('\n', '\\n').replace('\r', '\\r')
                    matches.append(f"Line {line_num}: {clean_line}")
                    if len(matches) >= 20:
                        break
            if not matches:
                return "No matches found"
            content = "\n".join(matches)
            if len(content) > 16 * 1024:
                content = content[:16 * 1024] + "... [truncated]"
            return content + f"\n--- File Info: {file_info['lines']} lines, {file_info['size']} bytes ---"
        except Exception as e:
            return f"Error processing file content {file_path}: {type(e).__name__} - {str(e)}"

    def get_parameters_schema(self, verbose: bool = True) -> Dict[str, Any]:
        """Return OpenAI-compatible schema for the tool."""
        try:
            schema = {
                "type": "function",
                "function": {
                    "name": "fs_read",
                    "description": "Discover files in tree structure or extract regex-matched snippets. Uses exclusions with fallback sorting." if verbose else "",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "operations": {
                                "type": "array",
                                "items": {
                                    "type": "object",
                                    "properties": {
                                        "mode": {
                                            "type": "string",
                                            "enum": [mode.value for mode in ToolCallMode],
                                            "description": "Select 'discover' to list files in tree or 'extract' to pull snippets." if verbose else ""
                                        },
                                        "path": {
                                            "type": "string",
                                            "description": "File or directory path relative to project root, e.g., 'src' or 'app.py'." if verbose else ""
                                        },
                                        "query": {
                                            "type": "string",
                                            "description": "Regex for fuzzy file name (DISCOVER) or content (EXTRACT). Required for EXTRACT." if verbose else ""
                                        },
                                        "file_pattern": {
                                            "type": "string",
                                            "description": "Glob filter, e.g., '*.py|*.ipynb' or '*.go|*.rs'." if verbose else ""
                                        },
                                        "max_depth": {
                                            "type": "integer",
                                            "description": "Maximum recursion depth for DISCOVER mode (default: 10)." if verbose else ""
                                        },
                                        "max_files": {
                                            "type": "integer",
                                            "description": "Maximum number of files to return in DISCOVER mode (default: 50)." if verbose else ""
                                        }
                                    },
                                    "required": ["mode", "path"]
                                }
                            }
                        },
                        "required": ["operations"]
                    }
                }
            }
            return schema["function"]["parameters"]
        except Exception as e:
            logger.error(f"Failed to generate parameters schema: {type(e).__name__} - {str(e)}")
            return {"error": f"Failed to generate parameters schema: {type(e).__name__} - {str(e)}"}

    def execute(self, **kwargs) -> Dict[str, Any]:
        """Execute filesystem read operations with robust error handling."""
        try:
            if 'params' in kwargs:
                params = kwargs['params']
            else:
                if 'operations' not in kwargs and ('mode' in kwargs or 'path' in kwargs):
                    kwargs = {'operations': [kwargs]}
                params = FsReadParams(**kwargs)
        except ValidationError as e:
            error_msg = f"Invalid parameters: {extract_validation_error(e)}"
            return create_error_response(error_msg)
        except Exception as e:
            return create_error_response(f"Parameter processing failed: {type(e).__name__} - {str(e)}")

        try:
            result = self._execute_internal(params)
            if len(params.operations) == 1 and result["success"]:
                op_result = result["data"][0]
                try:
                    json_data = json.loads(op_result["data"]) if op_result["data"] else {"data": [], "message": "Operation completed successfully"}
                    data = json_data.get("data", [])
                    message = json_data.get("message", "Operation completed successfully")
                    suggestions = json_data.get("suggestions", [])
                except json.JSONDecodeError as e:
                    return create_error_response(f"Failed to parse operation result: {type(e).__name__} - {str(e)}")
                return create_success_response(
                    message=message,
                    data=data,
                    processed_files=len(result["data"]),
                    suggestions=suggestions if suggestions else None
                )
            elif not result["success"]:
                op_result = result["data"][0] if result["data"] else {}
                error_data = op_result.get("error", {})
                message = error_data.get("message", "Operation failed") if isinstance(error_data, dict) else str(error_data)
                try:
                    json_data = json.loads(op_result.get("data", "{}"))
                    suggestions = json_data.get("suggestions", [])
                except (json.JSONDecodeError, KeyError):
                    suggestions = []
                return create_error_response(
                    message=message,
                    suggestions=suggestions if suggestions else None
                )
            return result
        except Exception as e:
            return create_error_response(f"Execution failed: {type(e).__name__} - {str(e)}")

    def _execute_internal(self, params: FsReadParams) -> Dict[str, Any]:
        """Internal execution logic for batch operations."""
        results = []
        total_processed = 0

        for op in params.operations:
            mode = op.mode.value
            path = op.path
            output_data = None
            error = None

            try:
                if mode == "discover":
                    output_data = self._discover_files(path, op.file_pattern, op.query or "", op.max_depth, op.max_files)
                    total_processed += op.max_files
                elif mode == "extract":
                    output_data = self._extract_content(path, op.query, op.file_pattern)
                    total_processed += 1
            except ValueError as e:
                error = {
                    "type": "PermissionError" if "outside project root" in str(e) else "ValueError",
                    "message": str(e)
                }
                logger.error(f"Error in {mode} on {path}: {str(e)}")
            except (OSError, PermissionError) as e:
                error = {"type": type(e).__name__, "message": str(e)}
                logger.error(f"OS error in {mode} on {path}: {str(e)}")
            except UnicodeDecodeError as e:
                error = {"type": "UnicodeDecodeError", "message": str(e)}
                logger.error(f"Decode error in {mode} on {path}: {str(e)}")
            except re.error as e:
                error = {"type": "RegexError", "message": str(e)}
                logger.error(f"Regex error in {mode} on {path}: {str(e)}")
            except Exception as e:
                error = {"type": type(e).__name__, "message": str(e)}
                logger.error(f"Unexpected error in {mode} on {path}: {type(e).__name__} - {str(e)}")

            results.append({
                "mode": mode,
                "path": path,
                "data": output_data,
                "error": error
            })

        overall_success = all(r.get("error") is None for r in results)
        overall_error = None if overall_success else {
            "type": "BatchError",
            "message": "Some operations failed—check individual errors for details"
        }
        return {
            "success": overall_success,
            "data": results,
            "error": overall_error,
            "metadata": {"processed_files": total_processed}
        }